In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [2]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, Tuple } from 'recursive-set';

In [3]:
import { RegExp as MyRegExp, RegExp2NFA, NFA } from "./03-RegExp-2-NFA";
import { parse } from "./RegExp-Parser";
import { nfa2dfa, DFA } from "./01-NFA-2-DFA";
import { nfa2dot, dfa2dot, dfa2string } from "./FSM-2-Dot";

In [4]:
function showSet<T>(s: RecursiveSet<T>): string {
    return `{${Array.from(s).map(String).sort().join(", ")}}`;
}

This notebook contains a test for the function `toNFA` implemented in the notebook `03-Regexp-2-NFA.ts`. In order to write regular expressions conveniently, we need the parser implemented in `RegExp-Parser`.


If the regular expression $r$ that is defined below has been written in the style of the lecture notes, it would read:
$$ (a \cdot b + b \cdot a)^* $$

In [5]:
const r = parse('(ab+ba)*');
r;

(((a, ⋅, b), +, (b, ⋅, a)), *)


In [6]:
const sigma = new RecursiveSet("a", "b");

In [7]:
const converter = new RegExp2NFA(sigma);

In [8]:
const nfa = converter.toNFA(r);

In [9]:
console.log(`States: ${showSet(nfa.Q)}`);
console.log(`Start: ${nfa.q0}`);
console.log(`Final: ${showSet(nfa.A)}`);

States: {1, 10, 11, 12, 2, 3, 4, 5, 6, 7, 8, 9}
Start: 11
Final: {12}


In [10]:
console.log("Transitions:");
for (const [key, targets] of nfa.delta) {
    console.log(`  (${key}) -> ${showSet(targets)}`);
}

Transitions:
  (1,a) -> {2}
  (3,b) -> {4}
  (2,ε) -> {3}
  (5,b) -> {6}
  (7,a) -> {8}
  (6,ε) -> {7}
  (9,ε) -> {1, 5}
  (4,ε) -> {10}
  (8,ε) -> {10}
  (11,ε) -> {12, 9}
  (10,ε) -> {12, 9}


In [11]:
const dotNFA = nfa2dot(nfa);

In [12]:
const viz = await instance();
display.html(viz.renderString(dotNFA, { format: "svg" }));

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.4 (20251115.1723)
 -->
<!-- Title: Non-Deterministic FSM Pages: 1 -->
 
 
 Non-Deterministic FSM 
 
<!-- 0 -->
 
 0 
 
 
<!-- 11 -->
 
 11 
 
 11 
 
<!-- 0->11 -->
 
 0->11 
 
 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 11->9 -->
 
 11->9 
 
 
 ε 
 
<!-- 12 -->
 
 12 
 
 
 12 
 
<!-- 11->12 -->
 
 11->12 
 
 
 ε 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 1->2 -->
 
 1->2 
 
 
 a 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 2->3 -->
 
 2->3 
 
 
 ε 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 3->4 -->
 
 3->4 
 
 
 b 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 4->10 -->
 
 4->10 
 
 
 ε 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 5->6 -->
 
 5->6 
 
 
 b 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 6->7 -->
 
 6->7 
 
 
 ε 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 7->8 -->
 
 7->8 
 
 
 a 
 
<!-- 8->10 -->
 
 8->10 
 
 
 ε 
 
<!-- 9->1 -->
 
 9->1 
 
 
 ε 
 
<!-- 9->5 -->
 
 9->5 
 
 
 ε 
 
<!-- 10->9 -->
 
 10->9 
 
 
 ε 
 
<!-- 10->12 -->
 
 10->12 
 
 
 ε

After having constructed a non-deterministic <span style="font-variant:small-caps;">Fsm</span> for the regular expression, we convert it into a deterministic <span style="font-variant:small-caps;">Fsm</span>.

In [13]:
const dfa = nfa2dfa(nfa);

In [14]:
dfa2string(dfa);

states: {S0, ..., S5}

start state: S3

state encoding:
S0 = {}
S1 = {2, 3}
S2 = {6, 7}
S3 = {1, 11, 12, 5, 9}
S4 = {1, 10, 12, 4, 5, 9}
S5 = {1, 10, 12, 5, 8, 9}

transitions:
delta(S0, a) = S0
delta(S0, b) = S0
delta(S1, a) = S0
delta(S1, b) = S4
delta(S2, a) = S5
delta(S2, b) = S0
delta(S3, a) = S1
delta(S3, b) = S2
delta(S4, a) = S1
delta(S4, b) = S2
delta(S5, a) = S1
delta(S5, b) = S2

set of accepting states: {S3, S4, S5}



In [15]:
const { dot: dotDFA, statesToNames } = dfa2dot(dfa);
display.html(viz.renderString(dotDFA, { format: "svg" }));

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.4 (20251115.1723)
 -->
<!-- Title: Deterministic FSM Pages: 1 -->
 
 
 Deterministic FSM 
 
<!-- 1 -->
 
 1 
 
 
<!-- S3 -->
 
 S3 
 
 
 S3 
 
<!-- 1->S3 -->
 
 1->S3 
 
 
 
<!-- S1 -->
 
 S1 
 
 S1 
 
<!-- S3->S1 -->
 
 S3->S1 
 
 
 a 
 
<!-- S2 -->
 
 S2 
 
 S2 
 
<!-- S3->S2 -->
 
 S3->S2 
 
 
 b 
 
<!-- S0 -->
 
 S0 
 
 S0 
 
<!-- S0->S0 -->
 
 S0->S0 
 
 
 a 
 
<!-- S0->S0 -->
 
 S0->S0 
 
 
 b 
 
<!-- S1->S0 -->
 
 S1->S0 
 
 
 a 
 
<!-- S4 -->
 
 S4 
 
 
 S4 
 
<!-- S1->S4 -->
 
 S1->S4 
 
 
 b 
 
<!-- S2->S0 -->
 
 S2->S0 
 
 
 b 
 
<!-- S5 -->
 
 S5 
 
 
 S5 
 
<!-- S2->S5 -->
 
 S2->S5 
 
 
 a 
 
<!-- S4->S1 -->
 
 S4->S1 
 
 
 a 
 
<!-- S4->S2 -->
 
 S4->S2 
 
 
 b 
 
<!-- S5->S1 -->
 
 S5->S1 
 
 
 a 
 
<!-- S5->S2 -->
 
 S5->S2 
 
 
 b

Note that the resulting DFA is not minimal, since we can identify some states.

We can inspect the mapping from DFA states (sets of NFA states) to their names:


In [ ]:
statesToNames;